* Join 이란? 한 데이터베이스 내의 여러 테이블의 레코드를 조합하여 하나의 열로 표현한 것 /
Join을 하면 서로 의미 있는 데이터들끼리 집합이 형성되기 때문에 각 테이블끼리 관계를 가지게 되고, 효율적인 검색과 결과를 얻을 수 있다.

- Inner Join (내부 조인) : 가장 흔한 결합 방식이며 쉽게 말해 ‘교집합’ 이라고 할 수 있다.

- Outer Join (외부 조인) : Left, Right, Full 조인으로 나눌 수 있으며 Full 을 제외하고는 각 왼쪽 테이블, 오른쪽 테이블 기준으로 데이터를 보여준다. 쉽게 말해 ‘합집합’ 이라고 할 수 있다.
만약 왼쪽 테이블을 기준으로 한다고 할때, 오른쪽 테이블에 해당하는 값이 없다면 Null로 채워진다.




In [ ]:
import pdb

def add_and_multiply(a, b):
    sum_value = a + b
    pdb.set_trace()
    product_value = a * b
    return sum_value, product_value

result_sum, result_product = add_and_multiply(3, 5)
print("The sum is:", result_sum)
print("The product is: ", result_product)

> <ipython-input-2-e42080907de5>(6)add_and_multiply()
      4     sum_value = a + b
      5     pdb.set_trace()
----> 6     product_value = a * b
      7     return sum_value, product_value
      8 

ipdb> sum_value
8
ipdb> n
> <ipython-input-2-e42080907de5>(7)add_and_multiply()
      5     pdb.set_trace()
      6     product_value = a * b
----> 7     return sum_value, product_value
      8 
      9 result_sum, result_product = add_and_multiply(3, 5)

ipdb> product_value
15
ipdb> c



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/bdb.py", line 347, in set_continue
    sys.settrace(None)



The sum is: 8
The product is:  15


In [ ]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt

In [ ]:
# 공유된 게시물 정보
articles_df = (
  pd.read_csv('/content/shared_articles.csv')
    .loc[lambda d: d['eventType'] == 'CONTENT SHARED']
)

In [ ]:
articles_df.head(5)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en
5,1459194522,CONTENT SHARED,-2826566343807132236,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en


In [ ]:
# 사용자 인터렉션 정보
interaction_df = pd.read_csv('/content/users_interactions.csv')
interaction_df.head(5)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN


### 데이터 전처리
- 각 액션에 대한 가중치 또는 강도 가정

In [ ]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0,
   'BOOKMARK': 2.5,
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,
}

interaction_df['eventStrength'] = (
  interaction_df
    .loc[:, 'eventType']
    .apply(lambda d: event_type_strength[d])
)

- 사용자에 대한 정보 부족 시, 추천 제공하기가 어려워지는 Cold Start 문제 존재
-> 따라서 5개 이상 인터렉션이 존재하는 사용자에 대한 데이터만 활용

In [ ]:
interaction_df_over5 = (interaction_df
  .groupby('personId', group_keys=False)
  .apply(lambda df: df.assign(interactCnt = lambda d: d['contentId'].nunique()))
  .loc[lambda d: d['interactCnt'] >= 5]
)

interaction_df_over5.shape[0]

69868

In [ ]:
interaction_full_df = (
  interaction_df_over5
    .groupby(['personId', 'contentId'], as_index=False)['eventStrength']
    .sum()
    .assign(eventScore = lambda d: np.log2(1+d['eventStrength']))
)

interaction_full_df.head(10)

,personId,contentId,eventStrength,eventScore
0,-9223121837663643404,-8949113594875411859,1.0,1.000000
1,-9223121837663643404,-8377626164558006982,1.0,1.000000
2,-9223121837663643404,-8208801367848627943,1.0,1.000000
3,-9223121837663643404,-8187220755213888616,1.0,1.000000
4,-9223121837663643404,-7423191370472335463,8.0,3.169925
5,-9223121837663643404,-7331393944609614247,1.0,1.000000
6,-9223121837663643404,-6872546942144599345,1.0,1.000000
7,-9223121837663643404,-6728844082024523434,1.0,1.000000
8,-9223121837663643404,-6590819806697898649,1.0,1.000000
9,-9223121837663643404,-6558712014192834002,2.0,1.584963


### 평가
- Cross Validatioin 기법 사용 (holdout)
- 임의로 추출한 20%의 테스트용 데이터 -> 평가를 위해서만 사용
- 정확도는 Top-N 정확도 사용 / 해당 유저가 상호작용한 적이 없는 상품 중 100개를 샘플링 / 추천 모형을 통해 상호작용한 상품과 상호작용 없었던 100개 상품에 대해 추천 순위 설정 / 이 사용자와 상품에 대한 Top-N 정확도를 계산

In [ ]:
interaction_train, interaction_test = train_test_split(
    interaction_full_df,
    stratify=interaction_full_df['personId'],
    test_size=0.2,
    random_state=42
)

interaction_full_indexed = interaction_full_df.set_index('personId')
interaction_train_indexed = interaction_train.set_index('personId')
interaction_test_indexed = interaction_test.set_index('personId')

def get_items_interacted(person_id, interaction_df):
    interated_items = interaction_df.loc[person_id]['contentId']
    return set(interated_items if type(interated_items) == pd.Series else [interated_items])

class ModelEvaluator:
    def __init__(self, n_non_interacted=100):
        self.EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = n_non_interacted

    def get_non_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, interaction_full_indexed)
        all_items = set(articles_df['contentId'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommend_items, topn):
        try:
            index = next(i for i, c in enumerate(recommend_items) if c == item_id)
        except:
            index = -1
        hit = int(index in range(0, topn))
        return hit, index

    def evaluate_model_for_user(self, model, person_id):
        interacted_values_testset = interaction_test_indexed.loc[person_id]
        if type(interacted_values_testset['contentId']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['contentId'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['contentId'])])

        interacted_items_count_testset = len(person_interacted_items_testset)

        # 특정 사용자에 대한 추천 순위 목록을 받아온다
        person_recs = model.recommend_items(
            person_id,
            items_to_ignore=get_items_interacted(person_id, interaction_train_indexed),
            topn=10000000000
        )

        hits_at_5_count = 0
        hits_at_10_count = 0

        # test셋에서 사용자가 상호작용한 모든 항목에 대해 반복한다
        for item_id in person_interacted_items_testset:

            # 사용자가 상호작용하지 않은 100개 항목을 샘플링한다
            non_interacted_items_sample = self.get_non_interacted_items_sample(
                person_id,
                sample_size=self.EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS,
                seed=item_id % (2**32)
            )

            # 현재 선택한 item_id(상호작용 있었던 항목)와 100개 랜덤 샘플을 합친다
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            # 추천 결과물 중에서 현재 선택한 item_id와 100개 랜덤 샘플의 결과물로만 필터링한다
            valid_recs_df = person_recs[person_recs['contentId'].isin(items_to_filter_recs)]
            valid_recs = valid_recs_df['contentId'].values

            # 현재 선택한 item_id가 Top-N 추천 결과 안에 있는지 확인한다
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        # Recall 값은 상호작용 있었던 항목들 중에서 관련없는 항목들과 섞였을 때 Top-N에 오른 항목들의 비율로 나타낼 수 있다
        recall_at_5 = hits_at_5_count / interacted_items_count_testset
        recall_at_10 = hits_at_10_count / interacted_items_count_testset

        person_metrics = {
            'hits@5_count': hits_at_5_count,
            'hits@10_count': hits_at_10_count,
            'interacted_count': interacted_items_count_testset,
            'recall@5': recall_at_5,
            'recall@10': recall_at_10
        }
        return person_metrics

    def evaluate_model(self, model):
        people_metrics = []
        for idx, person_id in enumerate(list(interaction_test_indexed.index.unique().values)):
            person_metrics = self.evaluate_model_for_user(model, person_id)
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)

        print('{} users processed'.format(idx))

        detailed_result = (
            pd.DataFrame(people_metrics)
              .sort_values('interacted_count', ascending=False)
        )

        global_recall_at_5 = detailed_result['hits@5_count'].sum() / detailed_result['interacted_count'].sum()
        global_recall_at_10 = detailed_result['hits@10_count'].sum() / detailed_result['interacted_count'].sum()

        global_metrics = {
            'model_name': model.get_model_name(),
            'recall@5': global_recall_at_5,
            'recall@10': global_recall_at_10
        }

        return global_metrics, detailed_result

model_evaluator = ModelEvaluator(n_non_interacted=100)

### Popularity Model
- 가장 기본적인 인기도 모형

In [ ]:
item_popularity = (interaction_full_df
 .groupby('contentId')['eventStrength'].sum()
 .sort_values(ascending=False)
 .reset_index()
)

item_popularity.head(10)

,contentId,eventStrength
0,-4029704725707465084,457.5
1,-2358756719610361882,361.0
2,-6783772548752091658,357.0
3,8657408509986329668,338.0
4,-133139342397538859,336.5
5,-8208801367848627943,327.5
6,-6843047699859121724,315.0
7,2581138407738454418,310.0
8,2857117417189640073,308.5
9,-1633984990770981161,298.0


In [ ]:
class PopularityRecommender:

    MODEL_NAME = 'Popularity'

    def __init__(self, popularity_df, items_df=None):
        self.popularity_df = popularity_df
        self.items_df = items_df

    def get_model_name(self):
        return self.MODEL_NAME

    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # 인기상품 중에서 사용자가 보지 않았던 상품을 추천한다
        recommendations = (
          self.popularity_df[~self.popularity_df['contentId'].isin(items_to_ignore)]
            .sort_values('eventStrength', ascending=False)
            .head(topn)
        )

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')
            recommendations = (recommendations
                .merge(self.items_df, how='left', left_on='contentId', right_on='contentId')
                .loc[:, ['eventStrength', 'contentId', 'title', 'url', 'lang']]
            )

        return recommendations
popularity_model = PopularityRecommender(item_popularity, articles_df)

- 인기도 모형 평가

In [ ]:
print('Popularity 추천 모형을 평가합니다')
pop_global_metrics, pop_detailed_results = model_evaluator.evaluate_model(popularity_model)
print('Global Metrics:\n{}'.format(pop_global_metrics))
pop_detailed_results.head(10)

Popularity 추천 모형을 평가합니다


<ipython-input-12-9b99fd1ebbf6>:26: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)


1139 users processed
Global Metrics:
{'model_name': 'Popularity', 'recall@5': 0.2241114804397852, 'recall@10': 0.3502940424443876}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
76,22,45,192,0.114583,0.234375,3609194402293569455
17,13,30,134,0.097015,0.223881,-2626634673110551643
16,14,32,130,0.107692,0.246154,-1032019229384696495
10,6,11,117,0.051282,0.094017,-1443636648652872475
82,24,35,88,0.272727,0.397727,-2979881261169775358
161,12,18,80,0.150000,0.225000,-3596626804281480007
65,15,26,73,0.205479,0.356164,1116121227607581999
81,14,19,69,0.202899,0.275362,692689608292948411
106,12,19,69,0.173913,0.275362,-9016528795238256703
52,20,29,68,0.294118,0.426471,3636910968448833585


### Content-Based Filtering Model
- 사용자가 상호작용했던 항목의 특성 바탕으로 유사한 항목 추천
- TF-IDF 사용 / 각 단어는 벡터 내에서 위치로 표현되고, 글에서 해당 단어가 몇 번 등장했는지를 수치로 나타냄

In [ ]:
import nltk
nltk.download('stopwords')

# stopwords를 제거한다
stopwords_list = stopwords.words('english') + stopwords.words('portuguese')

# 벡터의 길이는 5000, unigram과 bigram을 사용하고, stopwords를 제거하도록 학습한다
vectorizer = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1, 2),
    min_df=0.003,
    max_df=0.5,
    max_features=5000,
    stop_words=stopwords_list
)

item_ids = articles_df['contentId'].tolist()
tfidf_matrix = vectorizer.fit_transform(articles_df['title'] + '' + articles_df['text'])
# tfidf_feature_names = vectorizer.get_feature_names()
tfidf_feature_names = vectorizer.get_feature_names_out()
tfidf_matrix

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<3047x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 638118 stored elements in Compressed Sparse Row format>

해당 사용자가 상호작용했던 모든 항목들 특성의 평균을 계산 / 각 상호작용 강도에 따라 가중치 부여하여 평균 구함

In [ ]:
def get_item_profile(item_id):
    idx = item_ids.index(item_id)
    item_profile = tfidf_matrix[idx:idx+1]
    return item_profile

def get_item_profiles(ids):
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles

def build_user_profile(person_id, interaction_indexed_df):
    interaction_person_df = interaction_indexed_df.loc[person_id]
    user_item_profiles = get_item_profiles(interaction_person_df['contentId'])

    user_item_strengths = np.array(interaction_person_df['eventStrength']).reshape(-1, 1)

    # 상호작용 강도를 바탕으로 가중치를 부여하여 평균 계산한다
    user_item_strengths_weighted_avg = \
        np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) /\
        np.sum(user_item_strengths)

    user_item_strengths_weighted_avg = np.asarray(user_item_strengths_weighted_avg)  # 코드 추가

    user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)
    return user_profile_norm

def build_user_profiles():
    interaction_indexed_df = (interaction_full_df
        .loc[lambda d: d['contentId'].isin(articles_df['contentId'])]
        .set_index('personId')
    )
    user_profiles = {}

    for person_id in interaction_indexed_df.index.unique():
        user_profiles[person_id] = build_user_profile(person_id, interaction_indexed_df)

    return user_profiles

In [ ]:
user_profiles = build_user_profiles()
len(user_profiles)

1140

In [ ]:
myprofile = user_profiles[-1479311724257856983].flatten().tolist()
pd.DataFrame(sorted(zip(tfidf_feature_names, myprofile), key=lambda x: -x[1])[:20],
             columns=['token', 'relevance'])

,token,relevance
0,learning,0.312072
1,machine learning,0.269279
2,machine,0.257045
3,data,0.186686
4,google,0.171531
5,ai,0.141778
6,graph,0.114041
7,algorithms,0.109439
8,like,0.096131
9,language,0.085056


In [ ]:
class ContentBasedRecommender:

    MODEL_NAME = 'Content-Based'

    def __init__(self, item_ids, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df

    def get_model_name(self):
        return self.MODEL_NAME

    def _get_similar_items_to_user_profile(self, person_id, topn=1000):
        # 유저 특성과 항목 특성 사이의 코사인 유사도를 구한다
        cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)

        # 가장 유사한 항목을 찾는다
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]

        # 유사도를 기준으로 유사한 항목을 정렬한다
        similar_items = sorted(
            [(item_ids[i], cosine_similarities[0, i]) for i in similar_indices],
            key=lambda x: -x[1]
        )

        return similar_items

    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        similar_items = self._get_similar_items_to_user_profile(user_id)

        # 기존에 상호작용했던 항목은 제거한다
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))

        recommendations = (
            pd.DataFrame(similar_items_filtered, columns=['contentId', 'recStrength'])
              .head(topn)
        )

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')
            recommendations = (recommendations
                .merge(self.items_df, how='left', left_on='contentId', right_on='contentId')
                .loc[:, ['recStrength', 'contentId', 'title', 'url', 'lang']]
            )

        return recommendations
content_based_model = ContentBasedRecommender(item_ids, articles_df)

In [ ]:
print('콘텐츠 기반 추천 모형을 평가합니다')
cb_global_metrics, cb_detailed_results = model_evaluator.evaluate_model(content_based_model)
print('Global Metrics:\n{}'.format(cb_global_metrics))
cb_detailed_results.head(10)

콘텐츠 기반 추천 모형을 평가합니다


<ipython-input-12-9b99fd1ebbf6>:26: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
<ipython-input-12-9b99fd1ebbf6>:26: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
<ipython-input-12-9b99fd1ebbf6>:26: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
<ipython-input-12-9b99fd1ebbf6>:26: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
<ipython-input-12-9b99fd1ebbf6>:26: DeprecationWarning: Sampling from a set depr

1139 users processed
Global Metrics:
{'model_name': 'Content-Based', 'recall@5': 0.3863462030171312, 'recall@10': 0.4964203528509333}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
76,17,28,192,0.088542,0.145833,3609194402293569455
17,23,32,134,0.171642,0.238806,-2626634673110551643
16,21,35,130,0.161538,0.269231,-1032019229384696495
10,37,57,117,0.316239,0.487179,-1443636648652872475
82,8,15,88,0.090909,0.170455,-2979881261169775358
161,16,25,80,0.200000,0.312500,-3596626804281480007
65,10,16,73,0.136986,0.219178,1116121227607581999
81,11,20,69,0.159420,0.289855,692689608292948411
106,5,14,69,0.072464,0.202899,-9016528795238256703
52,5,10,68,0.073529,0.147059,3636910968448833585


### Matric Factorization

In [ ]:
users_items_pivot_df = (interaction_train
  .pivot(index='personId', columns='contentId', values='eventStrength')
  .fillna(0)
)
users_items_pivot_df.iloc[:5, :5]

contentId,-9222795471790223670,-9216926795620865886,-9194572880052200111,-9192549002213406534,-9190737901804729417
personId,,,,,
-9223121837663643404,0.0,0.0,0.0,0.0,0.0
-9212075797126931087,0.0,0.0,0.0,0.0,0.0
-9207251133131336884,0.0,3.0,0.0,0.0,0.0
-9199575329909162940,0.0,0.0,0.0,0.0,0.0
-9196668942822132778,0.0,0.0,0.0,0.0,0.0


In [ ]:
users_items_pivot_matrix = users_items_pivot_df.values
users_items_pivot_matrix[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 3., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
user_ids = list(users_items_pivot_df.index)
user_ids[:10]

[-9223121837663643404,
 -9212075797126931087,
 -9207251133131336884,
 -9199575329909162940,
 -9196668942822132778,
 -9188188261933657343,
 -9172914609055320039,
 -9156344805277471150,
 -9120685872592674274,
 -9109785559521267180]

In [ ]:
# User-Item matrix에서 요인의 개수를 정한다
NUMBER_OF_FACTORS_MF = 15

# User-Item Matrix을 분해한다
U, sigma, Vt = svds(users_items_pivot_matrix, k=NUMBER_OF_FACTORS_MF)

U.shape # (1140, 15)
Vt.shape # (15, 2926)

sigma_mat = np.diag(sigma)
sigma_mat.shape # (15, 15)

(15, 15)

In [ ]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma_mat), Vt)
all_user_predicted_ratings

array([[-3.59736043e-03,  1.13569387e-03,  1.20882188e-02, ...,
         1.92702938e-02,  5.12834501e-03,  2.51643553e-03],
       [ 9.71055720e-05,  8.75622910e-05,  3.93275556e-03, ...,
         4.06961661e-04, -4.19667479e-03, -6.14562512e-04],
       [-2.52746201e-02,  4.07052544e-03, -1.36013462e-02, ...,
         1.51481593e-02, -5.52722947e-03,  7.65220362e-03],
       ...,
       [ 2.07700031e-02,  1.02281302e-02, -5.38400117e-02, ...,
         2.87303379e-02, -5.95440040e-02, -3.52231861e-03],
       [-4.30766935e-02,  4.41158703e-03,  4.56109314e-02, ...,
         2.21704728e-02,  5.25493080e-04,  1.05746293e-03],
       [ 1.55442912e-02,  2.01216659e-02,  3.37166289e-01, ...,
         2.20774211e-03,  8.01526416e-02,  1.87568874e-02]])

In [ ]:
cf_preds_df = (
  pd.DataFrame(all_user_predicted_ratings,
               columns=users_items_pivot_df.columns,
               index=user_ids)
    .transpose()
)
cf_preds_df.iloc[:5, :5]

,-9223121837663643404,-9212075797126931087,-9207251133131336884,-9199575329909162940,-9196668942822132778
contentId,,,,,
-9222795471790223670,-0.003597,0.000097,-0.025275,0.049606,-0.014388
-9216926795620865886,0.001136,0.000088,0.004071,-0.000690,0.001928
-9194572880052200111,0.012088,0.003933,-0.013601,-0.006128,0.035771
-9192549002213406534,0.028340,-0.003469,-0.025762,-0.006675,0.011599
-9190737901804729417,0.015525,-0.001208,0.008202,0.001825,-0.000400


In [ ]:
class CFRecommender:

    MODEL_NAME = 'Collaborative Filtering'

    def __init__(self, cf_predictions_df, items_df=None):
        self.cf_predictions_df = cf_predictions_df
        self.items_df = items_df

    def get_model_name(self):
        return self.MODEL_NAME

    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # 사용자에 대한 예측값을 가져와서 정렬한다
        sorted_user_prediction = (self.cf_predictions_df
            .loc[:, user_id]
            .sort_values(ascending=False)
            .reset_index()
            .rename(columns={user_id: 'recStrength'})
        )

        recommendations = (sorted_user_prediction
            .loc[lambda d: ~d['contentId'].isin(items_to_ignore)]
            .sort_values('recStrength', ascending=False)
            .head(topn)
        )

        if verbose:
            if self.item_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations = (recommendations
                .merge(self.items_df, how='left', left_on='contentId', right_on='contentId')
                .loc[:, ['recStrength', 'contentId', 'title', 'url', 'lang']]
            )

        return recommendations
cf_recommender_model = CFRecommender(cf_preds_df, articles_df)

*** CF모형 평가 결과 -> 인기도 모형 보다는 높지만, 콘텐츠 기반 모형보다는 낮음

---

현재 데이터셋에서는 텍스트로부터 얻는 정보가 사용자 선호도를 모델링하는 데 더 도움이 되는 것 확인 가능


In [ ]:
print('협업 필터링(SVD 행렬분해) 모형을 평가합니다')
cf_global_metrics, cf_detailed_results = model_evaluator.evaluate_model(cf_recommender_model)
print('Global Metrics:\n{}'.format(cf_global_metrics))
cf_detailed_results.head(10)

협업 필터링(SVD 행렬분해) 모형을 평가합니다


<ipython-input-12-9b99fd1ebbf6>:26: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
<ipython-input-12-9b99fd1ebbf6>:26: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
<ipython-input-12-9b99fd1ebbf6>:26: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
<ipython-input-12-9b99fd1ebbf6>:26: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
<ipython-input-12-9b99fd1ebbf6>:26: DeprecationWarning: Sampling from a set depr

1139 users processed
Global Metrics:
{'model_name': 'Collaborative Filtering', 'recall@5': 0.2689849143441575, 'recall@10': 0.39682945538225517}


<ipython-input-12-9b99fd1ebbf6>:26: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
<ipython-input-12-9b99fd1ebbf6>:26: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
<ipython-input-12-9b99fd1ebbf6>:26: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
76,22,42,192,0.114583,0.218750,3609194402293569455
17,22,35,134,0.164179,0.261194,-2626634673110551643
16,26,36,130,0.200000,0.276923,-1032019229384696495
10,33,48,117,0.282051,0.410256,-1443636648652872475
82,36,47,88,0.409091,0.534091,-2979881261169775358
161,17,28,80,0.212500,0.350000,-3596626804281480007
65,15,25,73,0.205479,0.342466,1116121227607581999
81,14,21,69,0.202899,0.304348,692689608292948411
106,16,24,69,0.231884,0.347826,-9016528795238256703
52,17,29,68,0.250000,0.426471,3636910968448833585


### Hybrid Recommender
- 성능이 뛰어나 많이 사용됨

In [ ]:
class HybridRecommender:

    MODEL_NAME = 'Hybrid'

    def __init__(self, cb_rec_model, cf_rec_model, items_df):
        self.cb_rec_model = cb_rec_model
        self.cf_rec_model = cf_rec_model
        self.items_df = items_df

    def get_model_name(self):
        return self.MODEL_NAME

    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # 상위 1000개의 컨텐츠 기반 모형 추천을 가져온다
        cb_recs = (self.cb_rec_model
            .recommend_items(user_id, items_to_ignore=items_to_ignore, verbose=verbose, topn=1000)
            .rename(columns={'recStrength': 'recStrengthCB'})
        )

        # 상위 1000개의 협업필터링 추천을 가져온다
        cf_recs = (self.cf_rec_model
            .recommend_items(user_id, items_to_ignore=items_to_ignore, verbose=verbose, topn=1000)
            .rename(columns={'recStrength': 'recStrengthCF'})
        )

        # 1) 두 모형의 결과를 합친다
        # 2) CF, CB 모형의 점수를 바탕으로 Hybrid 모형의 점수를 계산한다
        # 3) Hybrid 점수를 기준으로 정렬한다
        recommendations = (cb_recs
            .merge(cf_recs, how='inner', left_on='contentId', right_on='contentId')
            .assign(recStrengthHybrid = lambda d: d['recStrengthCB'] * d['recStrengthCF'])
            .sort_values('recStrengthHybrid', ascending=False)
            .head(topn)
        )

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations = (recommendations
                .merge(self.items_df, how='left', left_on='contentId', right_on='contentId')
                .loc[:, ['recStrengthHybrid', 'contentId', 'title', 'url', 'lang']]
            )

        return recommendations
hybrid_recommender_model = HybridRecommender(content_based_model, cf_recommender_model, articles_df)

*** Recall@5가 0.3834, Recall@10은 0.4975

In [ ]:
print('하이브리드 모형을 평가합니다')
hybrid_global_metrics, hybrid_detailed_results = model_evaluator.evaluate_model(hybrid_recommender_model)
print('Global Metrics:\n{}'.format(hybrid_global_metrics))
hybrid_detailed_results.head(10)

하이브리드 모형을 평가합니다


<ipython-input-12-9b99fd1ebbf6>:26: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
<ipython-input-12-9b99fd1ebbf6>:26: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
<ipython-input-12-9b99fd1ebbf6>:26: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
<ipython-input-12-9b99fd1ebbf6>:26: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
<ipython-input-12-9b99fd1ebbf6>:26: DeprecationWarning: Sampling from a set depr

1139 users processed
Global Metrics:
{'model_name': 'Hybrid', 'recall@5': 0.38225517770391204, 'recall@10': 0.4953975965226285}


<ipython-input-12-9b99fd1ebbf6>:26: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
<ipython-input-12-9b99fd1ebbf6>:26: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
76,26,39,192,0.135417,0.203125,3609194402293569455
17,23,41,134,0.171642,0.305970,-2626634673110551643
16,26,38,130,0.200000,0.292308,-1032019229384696495
10,39,55,117,0.333333,0.470085,-1443636648652872475
82,26,32,88,0.295455,0.363636,-2979881261169775358
161,21,26,80,0.262500,0.325000,-3596626804281480007
65,15,24,73,0.205479,0.328767,1116121227607581999
81,13,20,69,0.188406,0.289855,692689608292948411
106,8,17,69,0.115942,0.246377,-9016528795238256703
52,14,22,68,0.205882,0.323529,3636910968448833585


In [ ]:
pd.DataFrame([pop_global_metrics, cf_global_metrics,
              cb_global_metrics, hybrid_global_metrics])

,model_name,recall@5,recall@10
0,Popularity,0.224111,0.350294
1,Collaborative Filtering,0.268985,0.396829
2,Content-Based,0.386346,0.496420
3,Hybrid,0.382255,0.495398
